## Import modules

In [1]:
import sys
sys.path.append('src')

In [2]:
import pandas
import numpy

from energiapy.components import *
from energiapy.utils.data_utils import make_henry_price_df, remove_outliers
from itertools import product

## Data Import

The following data is needed for the model

- solar and wind profiles : energiapy.fetch_nsrdb_data imports data from the NREL NSRDB database
- power demand : ERCOT for Houston; CAISO for San Diego
- Natural Gas prices: Henry Hub Price Index for both


**Get Weather data**

In [3]:
def load_data(loc:str, index:list):
    df = pandas.read_csv(f'data/{loc}_solar19.csv')
    df['idx'] = index
    df = df.set_index('idx')
    return df

In [4]:
idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]


In [5]:
weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]


**Demand data for San Diego (CAISO for SDGE region) and Houston (ERCOT for COAST region)**

In [6]:
demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
demand_ny = pandas.DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga

**Natural gas prices from Henry Hub Price Index**  

We use the special function energiapy.make_henry_price_df because of special requirements such as filling in empty data points (weekends, public holidays). Such function will be generalized in the next update of energipy

In [7]:
ng_price = make_henry_price_df(
    file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
ng_price = ng_price.drop(columns= 'scales')
ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
ng_price = pandas.concat([ng_price])

## Define temporal scale


The variabilities of energy systems are best captured over a discretized spatio-temporal scale. In energiapy, the first declaration is the temporal scale. 

For e.g.: Here we declare three temporal scales at different levels from right to left. The interpretation of these scales is merely symentic. Scales can be declared as the problem demands.
- 0, annual, with 1 discretization
- 1, daily with 365 discretization
- 2, hourly with 24 discretization

In essence, we are creating a temporal scale of 8760 points.

In [8]:
scales = TemporalScale(discretization_list=[1, 365, 24], design_scale= 0, scheduling_scale=2)

In [9]:
scales.ctype

[<ScaleType.MULTI: 1>, <ScaleType.SCHEDULING: 4>]

In [10]:
bigM = 10**6

## Declare resources

Resources can be consumed, produced, stored, discharged (or sold). 

In [11]:
th_h2 = Theta((0, 10))

In [12]:
Hydrogen = Resource(name='H2', discharge = True, sell_price = Theta((0, 10)))

In [13]:
Solar = Resource(name='Solar', gwp =50, eutt = 60, cons_max=100, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', cons_max= 100, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', discharge= True, label='Power generated')

Uranium = Resource(name='Ur', cons_max=100, purchase_price= 42.70/(250/2), basis='kg', label='Uranium')

H2 = Resource(name='H2', basis='tons', label='Hydrogen')

H2_L = Resource(name='H2_L', store_max = 10000, basis='tons', label='Hydrogen')

CO2_AQoff = Resource(name='CO2(Aq)', store_max = 10000, basis='tons', label='Carbon dioxide - sequestered')

H2O = Resource(name='H2O', cons_max=100,
               purchase_price= 0.001, basis='tons', label='Water')

CH4 = Resource(name='CH4', cons_max=100, purchase_price=1, basis='tons', label='Natural gas')

CO2 = Resource(name='CO2', basis='tons', label='Carbon dioxide', block='Resource')

CO2_Vent = Resource(name='CO2_Vent', discharge= True, basis='tons', label='Carbon dioxide - Vented', sell_price= 20)

O2 = Resource(name='O2', discharge=True, basis='tons', label='Oxygen')

CO2_DAC = Resource(name='CO2_DAC', basis='tons', label='Carbon dioxide - captured')


## Declare processes

In [14]:
LiI = Process(name='LiI', storage= Power, capex = 3516428, fopex= 87910, vopex = 0, store_max = 10000, cap_max=1000, label='Lithium-ion battery', basis = 'MW')

# WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= (0, 20), fopex= (0, 3), vopex=(0, 1), cap_max= Theta((1, 2)), land = Theta((4, 10)), label='Wind mill array', basis = 'MW')

WF = Process(name='WF', gwp = 50, odp = 2900, conversion={Wind: -1, Power: 1},capex=1462000, fopex=43000, vopex=4953, cap_max=1000, land = 30, label='Wind mill array', basis = 'MW')

# PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex = 1333262, fopex= 22623, cap_max=1000, label = 'Solar PV', basis = 'MW')
PV = Process(name='PV', conversion={1:{Solar: -1, Power: 0.8}, 2: {Solar: -1, Power: 1}}, capex = 1333262, fopex= 22623, cap_max=1000, label = 'Solar PV', basis = 'MW')

SMRH = Process(name='SMRH', conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 1.03, CO2: 9.332}, capex =2520000, fopex = 945000, vopex = 51.5, cap_max= 1000, label='Steam methane reforming + CCUS')

NGCC = Process(name= 'NGCC', conversion = {Power: 1, CH4: -0.108, CO2_Vent: 0.297*0.05, CO2:0.297*0.95}, capex = 2158928, fopex= 53320, vopex = 4090, cap_max= 1000, label = 'NGCC + 95% CC')

SMR = Process(name='SMR', capex = 2400, fopex = 800, vopex = 0.03,  conversion={Power: -1.11, CH4: -3.76, H2O: -23.7, H2: 1, CO2_Vent: 9.4979}, cap_max=1000, label='Steam methane reforming')

H2FC = Process(name='H2FC', conversion = {H2:-0.050, Power: 1}, capex =  1.6*10**6, vopex = 3.5, cap_max = 1000, label = 'hydrogen fuel cell')

DAC = Process(name='DAC', capex = 0.02536, fopex = 0.634, conversion={Power: -0.193, H2O: -4.048, CO2_DAC: 1}, cap_max=1000, label='Direct air capture')

PSH = Process(name='PSH', storage= Power, capex = 3924781, fopex= 17820, vopex = 512.5, store_max = 10000, cap_max=1000, label='Pumped storage hydropower', basis = 'MW')

H2_L_c = Process(name='H2_L_c', conversion={Power: -0.417, H2_L: 1, H2: -1}, capex =  1.6*10**6, vopex = 3.5, cap_max= 1000, label='Hydrogen geological storage')

H2_L_d = Process(name='H2_L_d', conversion={H2_L: -1, H2: 1}, capex =  0.01, vopex = 0.001, cap_max= 1000, label='Hydrogen geological storage discharge')

DAC = Process(name='DAC', conversion={Power: -0.193, H2O: -4.048, CO2_DAC: 1}, capex = 730, fopex= 114, vopex= 3.6, cap_max=1000, label='Direct air capture')

ASMR = Process(name='ASMR', conversion={Uranium: -4.17*10**(-5), H2O: -3.364, Power: 1}, capex = 7988951, fopex= 0.04*0.730, cap_max=1000, label='Small modular reactors (SMRs)')

AWE = Process(name='AWE', land = (0,20),  conversion={Power: -1, H2: 0.019, O2: 0.7632, H2O: -0.1753}, capex = 1.1*10**6, fopex = 16918, cap_max=1000, label='Alkaline water electrolysis (AWE)', citation='Demirhan et al. 2018 AIChE paper') 

AQoff_SMR = Process(name='AQoff_SMR', conversion={Power: -1.28, CO2_AQoff: 1, CO2: -1}, capex =  5.52, vopex = 4.14, cap_max= 1000,  label='Offshore aquifer CO2 sequestration (SMR)')

In [15]:
Hydrogen.sell_price

Theta(H2,sell_price)

In [16]:
process_list = {LiI, WF, PV, SMRH, SMR, DAC, AWE, H2_L_c, H2_L_d, AQoff_SMR, ASMR, PSH, NGCC}

## Declare locations

In [17]:
cc = Factor(data = pandas.DataFrame({'a': [2]}), scales= scales, apply_max_scaler= False)

houston = Location(name='HO', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, credit = {PV: 2}, cap_max_localize = {WF: 0.4, AWE: 23}, 
                   credit_factor ={PV: pandas.DataFrame({'a': [2]})}, cons_max_localize = {Solar: 2}, demand= {Power: (0, 1)}, 
                   land_cost = 10, land_cost_factor= cc, capacity_factor={PV: weather_ho[['dni']], WF: weather_ho[['wind_speed']]}, demand_factor={
                   Power: demand_ho}, purchase_price_factor={CH4: ng_price}, scales=scales, label='Houston')

sandiego = Location(name='SD', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, purchase_price_localize = {CH4: 222}, 
                    cons_max_localize = {Solar: 3}, land_cost = 30, land_cost_factor = pandas.DataFrame({'a': [2]}) , 
                    credit = {SMRH: 30, AQoff_SMR: Theta((0, 23))}, credit_factor= {SMRH: pandas.DataFrame({'a': [1 - 0.001*i for i in range(365)]})}, 
                    capacity_factor={PV: weather_sd[['dni']], WF: weather_sd[['wind_speed']]}, demand_factor={
                    Power: demand_sd}, purchase_price_factor={CH4: ng_price},  scales=scales, label='SanDiego')

newyork = Location(name='NY', processes={LiI, H2FC}, land_max = (0, 50000), land_max_factor = pandas.DataFrame({'a': [2]}), cap_max_localize = {PV: 0.2, AWE: 34}, 
                   capacity_factor={PV: weather_ny[['dni']], WF: weather_ny[['wind_speed']]}, demand= {Power: 40}, demand_factor={
                   Power: demand_ny}, credit = {PV: 7}, purchase_price_localize = {CH4: 2}, scales=scales, label='NewYork')


In [18]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)
    

## Declare transports

In [19]:

# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(name='Train_H2', resources={
                     H2}, cap_max=bigM, trans_loss=0.03, capex = 1000, vopex=1.667, fopex = (0,1), acid = 34, eutf =55, label='Railway for hydrogen transportation')
Grid = Transport(name='Grid', resources={
                 Power}, cap_max=bigM, trans_loss=0.001, capex = 500, vopex = 0.444, label='Railroad transport')



## Declare Network

In [20]:
distance_matrix = [
    [2366], 
    [2620]
]

transport_matrix = [
    [[Train_H2]], 
    [[Grid]],
]
capacity_factor  = {(houston, newyork): {Train_H2: weather_sd[['wind_speed']]}, (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]}}

network = Network(name='network', scales = scales, land_max= (0,5), land_max_factor= pandas.DataFrame({'a': [5]}),  
                  capacity_factor= capacity_factor, vopex_factor = {(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}}, sources=[houston, sandiego],
                  sinks=[newyork], distance_matrix=distance_matrix, transport_matrix=transport_matrix)


In [21]:

scenario = Scenario(name='scenario_full', network=network, scales=scales, label='full_case')


In [22]:
scenario.resources_locations_demand

{Power: {HO, NY, SD}}

In [23]:
scenario.resources_locations_discharge

{CO2_Vent: {HO, SD}, Power: {HO, NY, SD}, O2: {HO, SD}}

In [24]:
scenario.processes_locations_expenditure

{AWE: {HO, SD},
 PV: {HO, SD},
 SMRH: {HO, SD},
 LiI: {HO, NY, SD},
 H2FC: {NY},
 WF: {HO, SD},
 AQoff_SMR: {HO, SD}}

In [25]:
scenario.resources_

AttributeError: 'Scenario' object has no attribute 'resources_locations_sell'

In [24]:
houston.resources_sell

{CO2_Vent}

In [ ]:
scenario.collect_component_parameters

In [ ]:
scenario.resources_consume

{CH4, H2O, Solar, Wind}

In [ ]:
houston.resources_consume

{CH4, H2O, Solar, Wind}

In [ ]:
{i: i.resources_consume for i in scenario.locations if hasattr(i, 'resources_consume')}

{HO: {CH4, H2O, Solar, Wind}, SD: {CH4, H2O, Solar, Wind}}

{<ResourceParamType.AVAILABILITY: 3>: <ParameterType.CERTAIN: 1>,
 <ResourceParamType.PURCHASE_PRICE: 2>: <ParameterType.CERTAIN: 1>}

In [ ]:
resources_consume_locations = {resource: set() for resource in scenario.resources_consume}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in scenario.resources_consume}

In [ ]:
scenario.resources_consume

{CH4, H2O, Solar, Wind}

In [ ]:
resources_consume_locations = {resource: {location for location in self.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in self.resources_consume}

In [ ]:
resources_consume_locations 

{Solar: {HO, SD}, CH4: {HO, SD}, Wind: {HO, SD}, H2O: {HO, SD}}

In [ ]:
resources_demand_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_demand') and resource in location.resources_demand} for resource in scenario.resources_demand}

In [ ]:
scenario.resource_classifications()

{'CONSUME',
 'DEMAND',
 'DISCHARGE',
 'EMISSION',
 'IMPLICIT',
 'PRODUCE',
 'PURCHASE',
 'SELL',
 'STORE',
 'TRANSPORT'}

In [ ]:
{component: {location for location in getattr(self, 'locations') if hasattr(location, f'{component_set}_{parameter.lower()}') and resource in getattr(location, f'{component_set}_{parameter.lower()}')} for resource in getattr(self, f'{component_set}_{parameter.lower()}')}

In [ ]:
scenario.process_classifications()


{'CREDIT',
 'EMISSION',
 'EXPENDITURE',
 'FAILURE',
 'INTERMITTENT',
 'LAND',
 'LINEAR_CAPEX',
 'MULTI_MATMODE',
 'MULTI_PRODMODE',
 'NO_MATMODE',
 'PWL_CAPEX',
 'READINESS',
 'SINGLE_MATMODE',
 'SINGLE_PRODMODE',
 'STORAGE',
 'STORAGE_DISCHARGE',
 'STORAGE_REQ'}

In [ ]:
resources_demand_locations

{Power: {HO, NY, SD}}

In [ ]:
resources_demand_locations = {resource: {location for location in self.locations if hasattr(location, 'resources_demand') and resource in location.resources_demand} for resource in self.resources_demand}

In [119]:
i = 'ad_asd'.split('_')

In [121]:
i[1]

'asd'

In [109]:
resources_consume_locations

{Solar: {HO, SD}, CH4: {HO, SD}, Wind: {HO, SD}, H2O: {HO, SD}}

In [45]:
resources_consume_locations

{Solar: {HO, SD}, CH4: {HO, SD}, Wind: {HO, SD}, H2O: {HO, SD}}

In [100]:
from pyomo.environ import *
from energiapy.utils.scale_utils import scale_pyomo_set


In [101]:
m = ConcreteModel()
m.scales = Set(scenario.scales.list,
                    initialize=scenario.scales.scale)
m.scales_scheduling = scale_pyomo_set(
        instance=m, scale_level=scenario.scheduling_scale)
m.scales_design = scale_pyomo_set(
        instance=m, scale_level=scenario.design_scale)

In [94]:
# m.scales.pprint()
# m.scales_scheduling.pprint()

In [103]:
m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
m.b = Set(initialize = [(i.name, j.name) for i in resources_consume_locations for j in resources_consume_locations[i]])


In [104]:
m.a.pprint()

a : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     2 :    Any :    8 : {(Solar, HO), (Solar, SD), (CH4, HO), (CH4, SD), (Wind, HO), (Wind, SD), (H2O, HO), (H2O, SD)}


In [105]:
m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
m.B = Var(m.b, m.scales_design, within = NonNegativeReals)

In [106]:
m.A[Solar, houston, 0, 0, 1].pprint()

{Member of A} : Size=70080, Index=a*scales_scheduling
    Key                  : Lower : Value : Upper : Fixed : Stale : Domain
    (Solar, HO, 0, 0, 1) :     0 :  None :  None : False :  True : NonNegativeReals


In [107]:
m.B.pprint()

B : Size=8, Index=b*scales_design
    Key                : Lower : Value : Upper : Fixed : Stale : Domain
      ('CH4', 'HO', 0) :     0 :  None :  None : False :  True : NonNegativeReals
      ('CH4', 'SD', 0) :     0 :  None :  None : False :  True : NonNegativeReals
      ('H2O', 'HO', 0) :     0 :  None :  None : False :  True : NonNegativeReals
      ('H2O', 'SD', 0) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Solar', 'HO', 0) :     0 :  None :  None : False :  True : NonNegativeReals
    ('Solar', 'SD', 0) :     0 :  None :  None : False :  True : NonNegativeReals
     ('Wind', 'HO', 0) :     0 :  None :  None : False :  True : NonNegativeReals
     ('Wind', 'SD', 0) :     0 :  None :  None : False :  True : NonNegativeReals


In [ ]:
print(CH4.ftype)
print(PV.ftype)
print(sandiego.ftype)
print(Train_H2.ftype)
print(network.ftype)

{'RESOURCE_PURCHASE_PRICE': {HO, SD}}
{'PROCESS_CAPACITY': {HO, NY, SD, as}, 'PROCESS_CREDIT': {HO}}
{'LOCATION_LAND_COST'}
{'TRANSPORT_CAPACITY': {(SD, NY), (HO, NY)}, 'TRANSPORT_VOPEX': {(HO, SD)}}
{'NETWORK_LAND_MAX'}


In [ ]:
print(CH4.factors)
print(PV.factors)
print(sandiego.factors)
print(Train_H2.factors)
print(network.factors)

{'RESOURCE_PURCHASE_PRICE': {HO: Factor(CH4,HO,purchase_price), SD: Factor(CH4,SD,purchase_price)}}
{'PROCESS_CAPACITY': {HO: Factor(PV,HO,capacity), SD: Factor(PV,SD,capacity), NY: Factor(PV,NY,capacity), as: Factor(PV,as,capacity)}, 'PROCESS_CREDIT': {HO: Factor(PV,HO,credit)}}
{'LOCATION_LAND_COST': Factor(SD,land_cost)}
{'TRANSPORT_CAPACITY': {(HO, NY): Factor(Train_H2,(HO,NY),capacity), (SD, NY): Factor(Train_H2,(SD,NY),capacity)}, 'TRANSPORT_VOPEX': {(HO, SD): Factor(Train_H2,(HO,SD),vopex)}}
{'NETWORK_LAND_MAX': Factor(network,land_max)}


In [34]:
print(CH4.ltype)
print(PV.ltype)

{'RESOURCE_PURCHASE_PRICE': {NY, SD, as}}
{'PROCESS_CAP_MAX': {NY, as}}


In [35]:
print(CH4.localizations)
print(PV.localizations)

{'RESOURCE_PURCHASE_PRICE': {SD: Localize(CH4,SD,purchase_price), NY: Localize(CH4,NY,purchase_price), as: Localize(CH4,as,purchase_price)}}
{'PROCESS_CAP_MAX': {NY: Localize(PV,NY,cap_max), as: Localize(PV,as,cap_max)}}


In [36]:
PV.localizations['PROCESS_CAP_MAX'][newyork].ltype

'PROCESS_CAP_MAX'

In [37]:
print(scales.ctype)
print(Power.ctype)
print(PV.ctype)
print(sandiego.ctype)
print(Train_H2.ctype)
print(network.ctype)

[<ScaleType.MULTI: 1>, <ScaleType.SCHEDULING: 4>]
[<ResourceType.DISCHARGE: 4>, <ResourceType.PRODUCE: 2>, {<ResourceType.DEMAND: 8>: {HO, NY, as}}, <ResourceType.TRANSPORT: 9>, <ResourceType.TRANSPORT: 9>]
[<ProcessType.MULTI_PRODMODE: 2>, <ProcessType.NO_MATMODE: 3>, <ProcessType.LINEAR_CAPEX: 9>, <ProcessType.EXPENDITURE: 14>, {<ProcessType.CREDIT: 12>: {HO, NY, as}}, {<ProcessType.INTERMITTENT: 11>: {HO, NY, SD, as}}]
[<LocationType.LAND: 3>, <LocationType.SOURCE: 1>, <LocationType.SOURCE: 1>]
[<TransportType.NO_MATMODE: 1>, <TransportType.LINEAR_CAPEX: 4>, <TransportType.EXPENDITURE: 8>, <TransportType.EMISSION: 11>, {<TransportType.INTERMITTENT: 6>: {(SD, NY), (HO, NY)}}]
[<NetworkType.LAND: 1>]


In [38]:
print(Power.ptype)
print(PV.ptype)
print(sandiego.ptype)
print(Train_H2.ptype)
print(network.ptype)

{<ResourceParamType.DEMAND: 4>: {HO: <ParameterType.UNCERTAIN: 2>, NY: <ParameterType.UNCERTAIN: 2>, as: <ParameterType.UNCERTAIN: 2>}}
{<ProcessParamType.CAPACITY: 8>: <ParameterType.CERTAIN: 1>, <ProcessParamType.CAP_MAX: 1>: <ParameterType.CERTAIN: 1>, <ProcessParamType.FOPEX: 5>: <ParameterType.CERTAIN: 1>, <ProcessParamType.CAPEX: 4>: <ParameterType.CERTAIN: 1>, <ProcessParamType.CREDIT: 9>: {HO: <ParameterType.UNCERTAIN: 2>, NY: <ParameterType.UNCERTAIN: 2>, as: <ParameterType.UNCERTAIN: 2>}}
{<LocationParamType.LAND_COST: 1>: <ParameterType.CERTAIN: 1>}
{<TransportParamType.CAPACITY: 8>: <ParameterType.CERTAIN: 1>, <TransportParamType.TRANS_LOSS: 9>: <ParameterType.CERTAIN: 1>, <TransportParamType.VOPEX: 6>: <ParameterType.CERTAIN: 1>, <TransportParamType.CAP_MAX: 1>: <ParameterType.CERTAIN: 1>, <TransportParamType.FOPEX: 5>: <ParameterType.UNCERTAIN: 2>, <TransportParamType.CAPEX: 4>: <ParameterType.CERTAIN: 1>}
{<NetworkParamType.LAND_MAX: 1>: <ParameterType.UNCERTAIN: 2>}


In [40]:
houston.resources_demand

{Power}

In [53]:
scenario.data_network

{'distance_dict': {(HO, NY): 2366, (SD, NY): 2620},
 'transport_dict': {(HO, NY): {Train_H2}, (SD, NY): {Grid}},
 'land_max': Theta(network,land_max)}

In [54]:
AQoff_SMR.credit

{SD: Theta(AQoff_SMR,SD,credit)}

In [55]:
network.land_max

Theta(network,land_max)

In [56]:
a = ['as']

if a:
    print('asa')

asa


In [57]:
network.transport_avail_dict

{(HO, NY): {'Train_H2'}, (SD, NY): {'Grid'}}

In [58]:
network.transport_dict

{(HO, NY): {Train_H2}, (SD, NY): {Grid}}

In [59]:
network.distance_dict

{(HO, NY): 2366, (SD, NY): 2620}

In [60]:
list('as')

['a', 's']

In [61]:
a = set()

In [62]:
a.add('as')

In [63]:
a

{'as'}

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= houston, process= PV, fig_size= (9,5), color= 'orange')


![cap fac pv](plots/cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= sandiego, process= WF, fig_size= (9,5), color= 'blue')


![cap fac wf](plots/wf_sd.png)

In [ ]:
plot_scenario.demand_factor(scenario = scenario, location= newyork, resource= Power, fig_size= (9,5), color= 'red')


![df ny](plots/df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(scenario= scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.COST)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:

milp_cost.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_cost.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_cost = solve(scenario = scenario, instance= milp_cost, solver= 'gurobi', name="results_cost", print_solversteps = True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny](plots/sch_sdny.png)

In [ ]:
plot_results.transport(results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch hony](plots/sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.CAPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.VOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.FOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.TOTAL, location= 'HO', fig_size= (8,6))

![total cost ho](plots/total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:

plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.CAPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.VOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.FOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.TOTAL, fig_size= (8,6))

![total lw](plots/total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'Inv', component= 'LiI_Power_stored', location = 'SD', fig_size= (9,5), color = 'steelblue')

![inv lii](plots/lii_inv.png)

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'P', component= 'PV', location = 'SD', fig_size= (9,5), color = 'steelblue')

![pv p](plots/pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:

milp_demand = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                        Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.MAX_DISCHARGE, objective_resource=Power)


In [ ]:

milp_demand.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_demand.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_demand = solve(scenario=scenario, instance=milp_demand, solver='gurobi',
                       name="results_demand", print_solversteps=True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny2](plots/sch_sdny2.png)

In [ ]:
plot_results.transport(results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch t hony](plots/sch_hony2.png)